In [1]:
#라이브러리 임포트 
import pandas as pd
import numpy as np 
import optuna
from time import time
from tqdm import tqdm_notebook
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from sklearn.metrics import mean_absolute_error,accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier



In [2]:
path = 'D:\study_data\_data\_csv\dacon_travel/' 
train_set = pd.read_csv(path + 'train.csv',index_col=0)
test_set = pd.read_csv(path + 'test.csv', index_col=0)

경로 설정

In [3]:
print(train_set.info())
print(test_set.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1955 entries, 1 to 1955
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       1861 non-null   float64
 1   TypeofContact             1945 non-null   object 
 2   CityTier                  1955 non-null   int64  
 3   DurationOfPitch           1853 non-null   float64
 4   Occupation                1955 non-null   object 
 5   Gender                    1955 non-null   object 
 6   NumberOfPersonVisiting    1955 non-null   int64  
 7   NumberOfFollowups         1942 non-null   float64
 8   ProductPitched            1955 non-null   object 
 9   PreferredPropertyStar     1945 non-null   float64
 10  MaritalStatus             1955 non-null   object 
 11  NumberOfTrips             1898 non-null   float64
 12  Passport                  1955 non-null   int64  
 13  PitchSatisfactionScore    1955 non-null   int64  
 14  OwnCar  

dtypes: float64(7), int64(5), object(6) 데이터 타입을 확인합니다.
Object 타입인 ['TypeofContact','Occupation','Gender','ProductPitched','MaritalStatus','Designation'] 컬럼은 이후 학습을 위해 
인코딩이 필요합니다.


In [4]:
print(train_set.describe())

               Age     CityTier  DurationOfPitch  NumberOfPersonVisiting  \
count  1861.000000  1955.000000      1853.000000             1955.000000   
mean     37.462117     1.641432        15.524015                2.922762   
std       9.189948     0.908744         8.150057                0.712276   
min      18.000000     1.000000         5.000000                1.000000   
25%      31.000000     1.000000         9.000000                2.000000   
50%      36.000000     1.000000        14.000000                3.000000   
75%      43.000000     3.000000        20.000000                3.000000   
max      61.000000     3.000000        36.000000                5.000000   

       NumberOfFollowups  PreferredPropertyStar  NumberOfTrips     Passport  \
count        1942.000000            1945.000000    1898.000000  1955.000000   
mean            3.718332               3.568638       3.255532     0.291049   
std             1.004095               0.793196       1.814698     0.454362   

In [5]:
print(train_set.isnull().sum())
print(test_set.isnull().sum())
'''
컬럼 참고용 
id : 샘플 아이디
Age : 나이
TypeofContact : 고객의 제품 인지 방법 (회사의 홍보 or 스스로 검색)
CityTier : 주거 중인 도시의 등급. (인구, 시설, 생활 수준 기준) (1등급 > 2등급 > 3등급) 
DurationOfPitch : 영업 사원이 고객에게 제공하는 프레젠테이션 기간
Occupation : 직업
Gender : 성별
NumberOfPersonVisiting : 고객과 함께 여행을 계획 중인 총 인원
NumberOfFollowups : 영업 사원의 프레젠테이션 후 이루어진 후속 조치 수
ProductPitched : 영업 사원이 제시한 상품
PreferredPropertyStar : 선호 호텔 숙박업소 등급
MaritalStatus : 결혼여부
NumberOfTrips : 평균 연간 여행 횟수
Passport : 여권 보유 여부 (0: 없음, 1: 있음)
PitchSatisfactionScore : 영업 사원의 프레젠테이션 만족도
OwnCar : 자동차 보유 여부 (0: 없음, 1: 있음)
NumberOfChildrenVisiting : 함께 여행을 계획 중인 5세 미만의 어린이 수
Designation : (직업의) 직급
MonthlyIncome : 월 급여
ProdTaken : 여행 패키지 신청 여부 (0: 신청 안 함, 1: 신청함)
'''

Age                          94
TypeofContact                10
CityTier                      0
DurationOfPitch             102
Occupation                    0
Gender                        0
NumberOfPersonVisiting        0
NumberOfFollowups            13
ProductPitched                0
PreferredPropertyStar        10
MaritalStatus                 0
NumberOfTrips                57
Passport                      0
PitchSatisfactionScore        0
OwnCar                        0
NumberOfChildrenVisiting     27
Designation                   0
MonthlyIncome               100
ProdTaken                     0
dtype: int64
Age                         132
TypeofContact                15
CityTier                      0
DurationOfPitch             149
Occupation                    0
Gender                        0
NumberOfPersonVisiting        0
NumberOfFollowups            32
ProductPitched                0
PreferredPropertyStar        16
MaritalStatus                 0
NumberOfTrips              

'\n컬럼 참고용 \nid : 샘플 아이디\nAge : 나이\nTypeofContact : 고객의 제품 인지 방법 (회사의 홍보 or 스스로 검색)\nCityTier : 주거 중인 도시의 등급. (인구, 시설, 생활 수준 기준) (1등급 > 2등급 > 3등급) \nDurationOfPitch : 영업 사원이 고객에게 제공하는 프레젠테이션 기간\nOccupation : 직업\nGender : 성별\nNumberOfPersonVisiting : 고객과 함께 여행을 계획 중인 총 인원\nNumberOfFollowups : 영업 사원의 프레젠테이션 후 이루어진 후속 조치 수\nProductPitched : 영업 사원이 제시한 상품\nPreferredPropertyStar : 선호 호텔 숙박업소 등급\nMaritalStatus : 결혼여부\nNumberOfTrips : 평균 연간 여행 횟수\nPassport : 여권 보유 여부 (0: 없음, 1: 있음)\nPitchSatisfactionScore : 영업 사원의 프레젠테이션 만족도\nOwnCar : 자동차 보유 여부 (0: 없음, 1: 있음)\nNumberOfChildrenVisiting : 함께 여행을 계획 중인 5세 미만의 어린이 수\nDesignation : (직업의) 직급\nMonthlyIncome : 월 급여\nProdTaken : 여행 패키지 신청 여부 (0: 신청 안 함, 1: 신청함)\n'

train과 test 모두 ['Age','DurationOfPitch','NumberOfFollowups','PreferredPropertyStar',
'NumberOfTrips','NumberOfChildrenVisiting','NumberOfChildrenVisiting']의 컬럼이 결측이 존재합니다.

In [6]:
#결측치 처리['Age'] 
print(train_set[train_set['Age'].notnull()].groupby(['Designation'])['Age'].mean())
print(train_set[train_set['Age'].notnull()].groupby(['Occupation'])['Age'].mean())
train_set['Age'].fillna(train_set.groupby('Designation')['Age'].transform('mean'), inplace=True)
test_set['Age'].fillna(test_set.groupby('Designation')['Age'].transform('mean'), inplace=True)
print(train_set['Age'].value_counts())

Designation
AVP               48.153846
Executive         32.742775
Manager           37.235747
Senior Manager    41.235880
VP                47.426966
Name: Age, dtype: float64
Occupation
Free Lancer       37.000000
Large Business    36.761006
Salaried          37.614629
Small Business    37.426752
Name: Age, dtype: float64
35.000000    96
33.000000    88
32.000000    86
36.000000    84
37.000000    84
34.000000    79
38.000000    74
41.000000    74
30.000000    70
29.000000    68
31.000000    68
28.000000    62
39.000000    61
40.000000    57
42.000000    55
45.000000    53
32.742775    53
43.000000    53
27.000000    51
26.000000    42
46.000000    42
44.000000    41
37.235747    40
51.000000    36
49.000000    32
47.000000    31
25.000000    31
53.000000    30
50.000000    29
54.000000    27
48.000000    26
52.000000    24
24.000000    24
55.000000    23
23.000000    23
22.000000    20
21.000000    17
56.000000    17
19.000000    16
59.000000    14
20.000000    13
60.000000    12
5

나이의 결측은 직급별 평균의 나이로 채웁니다.직급과 직업 컬럼으로 평균으로 비교해보았습니다만 직업별 평균으로 값을 대체하기에는 
각 값별 편차가 적어 적절하지 않습니다.직급별 평균은 편차가 크기 때문에 값을 대체하기에 적절합니다.

In [7]:
#결측치 처리['TypeofContact']
print(train_set['TypeofContact'].value_counts())

train_set['TypeofContact'].fillna('Self Enquiry', inplace=True)
test_set['TypeofContact'].fillna('Self Enquiry', inplace=True)

Self Enquiry       1379
Company Invited     566
Name: TypeofContact, dtype: int64


['TypeofContact']의 컬럼은 결측치가 비교적 작기 때문에 최빈값인 Self Enquiry 로 대체합니다.

In [8]:
#결측치 처리 ['NumberOfChildrenVisiting','NumberOfFollowups']
print(train_set[train_set['NumberOfChildrenVisiting'].notnull()].groupby(['MaritalStatus'])['NumberOfChildrenVisiting'].mean())
print(train_set[train_set['NumberOfFollowups'].notnull()].groupby(['NumberOfChildrenVisiting'])['NumberOfFollowups'].mean())
train_set['NumberOfChildrenVisiting'].fillna(train_set.groupby('MaritalStatus')['NumberOfChildrenVisiting'].transform('mean'), inplace=True)
test_set['NumberOfChildrenVisiting'].fillna(test_set.groupby('MaritalStatus')['NumberOfChildrenVisiting'].transform('mean'), inplace=True)
train_set['NumberOfFollowups'].fillna(train_set.groupby('NumberOfChildrenVisiting')['NumberOfFollowups'].transform('mean'), inplace=True)
test_set['NumberOfFollowups'].fillna(test_set.groupby('NumberOfChildrenVisiting')['NumberOfFollowups'].transform('mean'), inplace=True)

MaritalStatus
Divorced     1.182561
Married      1.233227
Single       0.991176
Unmarried    1.453901
Name: NumberOfChildrenVisiting, dtype: float64
NumberOfChildrenVisiting
0.0    3.142857
1.0    3.776138
2.0    3.949550
3.0    4.164179
Name: NumberOfFollowups, dtype: float64


['NumberOfChildrenVisiting']컬럼은 결혼 여부에 따라 편차 있다고 생각하여 ['MaritalStatus']컬럼의 평균값으로 대체하였습니다.
['NumberOfFollowups']의 결측치는 비교적 평균값에 편차가 큰 컬럼의 평균값으로 대체하였습니다.


In [9]:
#결측치 처리 ['MonthlyIncome']

print(train_set[train_set['MonthlyIncome'].notnull()].groupby(['Designation'])['MonthlyIncome'].mean())

train_set['MonthlyIncome'].fillna(train_set.groupby('Designation')['MonthlyIncome'].transform('mean'), inplace=True)
test_set['MonthlyIncome'].fillna(test_set.groupby('Designation')['MonthlyIncome'].transform('mean'), inplace=True)

Designation
AVP               32148.438462
Executive         20110.209859
Manager           22614.373397
Senior Manager    26715.056291
VP                35796.179775
Name: MonthlyIncome, dtype: float64


['MonthlyIncome']의 결측치는 직급 컬럼인 ['Designation']과 비교했을 때 유의미한 차이를 보여 결측치를 대체하였습니다.

In [10]:
#결측치 처리 ['DurationOfPitch']

train_set['DurationOfPitch']=train_set['DurationOfPitch'].fillna(0)
test_set['DurationOfPitch']=test_set['DurationOfPitch'].fillna(0)

영업 사원이 고객에게 제공하는 프레젠테이션 기간은 특정 컬럼의 평균값으로 대체하기 보다는 '0'으로 대체하였습니다.

In [11]:
#결측치 처리 ['PreferredPropertyStar']
print(train_set[train_set['PreferredPropertyStar'].notnull()].groupby(['Occupation'])['PreferredPropertyStar'].mean())
train_set['PreferredPropertyStar'].fillna(train_set.groupby('Occupation')['PreferredPropertyStar'].transform('mean'), inplace=True)
test_set['PreferredPropertyStar'].fillna(test_set.groupby('Occupation')['PreferredPropertyStar'].transform('mean'), inplace=True)


Occupation
Free Lancer       3.000000
Large Business    3.564417
Salaried          3.551255
Small Business    3.590303
Name: PreferredPropertyStar, dtype: float64


해당 컬럼은 더 고려해야할 컬럼이 많을 수 있지만 결측의 양도 많지 않아 ['Occupation'] 컬럼의 평균값으로 대체하였습니다.

In [12]:
cut = [train_set,test_set]

for dataset in cut:    
    dataset.loc[ dataset['Age'] <= 20, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 20) & (dataset['Age'] <= 29), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 29) & (dataset['Age'] <= 39), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 39) & (dataset['Age'] <= 49), 'Age'] = 3
    dataset.loc[(dataset['Age'] > 49) & (dataset['Age'] <= 59), 'Age'] = 4
    dataset.loc[ dataset['Age'] > 59, 'Age'] = 5
    
print(train_set['Age'].value_counts())

2.0    883
3.0    465
1.0    338
4.0    220
0.0     34
5.0     15
Name: Age, dtype: int64


연속형 수치데이터인 ['Age'] 컬럼을 범주화해 범위를 줄였습니다.

In [13]:
#결측치 처리['NumberOfTrips']
print(train_set[train_set['NumberOfTrips'].notnull()].groupby(['DurationOfPitch'])['NumberOfTrips'].mean())

train_set['NumberOfTrips'].fillna(train_set.groupby('DurationOfPitch')['NumberOfTrips'].transform('mean'), inplace=True)
test_set['NumberOfTrips'].fillna(test_set.groupby('DurationOfPitch')['NumberOfTrips'].transform('mean'), inplace=True)

DurationOfPitch
0.0     2.813725
5.0     1.666667
6.0     2.612069
7.0     3.551724
8.0     3.139344
9.0     3.594595
10.0    3.343434
11.0    3.102564
12.0    3.256098
13.0    3.590361
14.0    3.311927
15.0    3.203883
16.0    2.909910
17.0    3.492958
18.0    3.304348
19.0    4.111111
20.0    4.586207
21.0    2.833333
22.0    2.818182
23.0    3.365854
24.0    2.884615
25.0    3.343750
26.0    2.851852
27.0    3.032258
28.0    3.208333
29.0    2.333333
30.0    3.025641
31.0    3.176471
32.0    3.900000
33.0    4.578947
34.0    3.833333
35.0    3.379310
36.0    3.714286
Name: NumberOfTrips, dtype: float64


['NumberOfTrips']의 결측치는 ['DurationOfPitch']의 평균으로 대체하였습니다.

In [14]:
print(train_set['Occupation'].value_counts())
train_set.loc[ train_set['Occupation'] =='Free Lancer' , 'Occupation'] = 'Salaried'
test_set.loc[ test_set['Occupation'] =='Free Lancer' , 'Occupation'] = 'Salaried'

print(train_set['Gender'].value_counts()) 
train_set.loc[ train_set['Gender'] =='Fe Male' , 'Gender'] = 'Female'
test_set.loc[ test_set['Gender'] =='Fe Male' , 'Gender'] = 'Female'

Salaried          965
Small Business    826
Large Business    163
Free Lancer         1
Name: Occupation, dtype: int64
Male       1207
Female      692
Fe Male      56
Name: Gender, dtype: int64


['Occupation']의 Free Lancer는 결측치는 아니지만 이상치로서 최빈값인 'Salaried'로 대체하였습니다.
['Gender']는 성별이므로 남,여 이외의 성이 있을 수 없으므로 Fe Male은 오타로 보여 'Female'로 대체하였습니다.

In [15]:
# Object 컬럼 처리 
columns = ['TypeofContact','Occupation','Gender','ProductPitched','MaritalStatus','Designation']
for col in tqdm_notebook(columns):
    le = LabelEncoder()
    train_set[col]=le.fit_transform(train_set[col])
    test_set[col]=le.transform(test_set[col])

C:\Users\BORI\AppData\Local\Temp\ipykernel_2428\3759354840.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for col in tqdm_notebook(columns):


  0%|          | 0/6 [00:00<?, ?it/s]

In [16]:
print(train_set.isnull().sum())
print(test_set.isnull().sum())

Age                         0
TypeofContact               0
CityTier                    0
DurationOfPitch             0
Occupation                  0
Gender                      0
NumberOfPersonVisiting      0
NumberOfFollowups           0
ProductPitched              0
PreferredPropertyStar       0
MaritalStatus               0
NumberOfTrips               0
Passport                    0
PitchSatisfactionScore      0
OwnCar                      0
NumberOfChildrenVisiting    0
Designation                 0
MonthlyIncome               0
ProdTaken                   0
dtype: int64
Age                         0
TypeofContact               0
CityTier                    0
DurationOfPitch             0
Occupation                  0
Gender                      0
NumberOfPersonVisiting      0
NumberOfFollowups           0
ProductPitched              0
PreferredPropertyStar       0
MaritalStatus               0
NumberOfTrips               0
Passport                    0
PitchSatisfactionScore     

train과 test의 결측치가 사라진 것을 확인하였습니다.

In [17]:
x = train_set.drop(['ProdTaken','NumberOfChildrenVisiting','NumberOfPersonVisiting','OwnCar', 'MonthlyIncome', 'NumberOfFollowups'], axis=1)
test_set = test_set.drop(['NumberOfChildrenVisiting','NumberOfPersonVisiting','OwnCar','MonthlyIncome','NumberOfFollowups'], axis=1)
y = train_set['ProdTaken']
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.91,shuffle=True,random_state=1234,stratify=y)


train에 데이터를 x로 target은 y로 할당합니다.아이러니하게도 train과 test에서 해당 컬럼들을 drop하는 것이 모델 성능 개선에 도움이 되어 
drop을 실시하였습니다.

In [18]:

def objectiveCAT(trial: Trial, x_train, y_train, x_test):
    param = {
        'n_estimators' : trial.suggest_int('n_estimators', 300, 3500),
        'depth' : trial.suggest_int('depth', 6, 14),
        'fold_permutation_block' : trial.suggest_int('fold_permutation_block', 1, 256),
        'learning_rate' : trial.suggest_float('learning_rate', 0, 1),
        'od_pval' : trial.suggest_float('od_pval', 0, 1),
        'l2_leaf_reg' : trial.suggest_float('l2_leaf_reg', 0, 4),
    }
    
    # 학습 모델 생성
    model = CatBoostClassifier(**param)
    CAT_model = model.fit(x_train, y_train, verbose=True) 
    
    # 모델 성능 확인
    score = accuracy_score(CAT_model.predict(x_test), y_test)
    
    return score
study = optuna.create_study(direction='maximize', sampler=TPESampler())

study.optimize(lambda trial : objectiveCAT(trial, x, y, x_test), n_trials = 5)

print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))
# Best trial : score 1.0,
# params {'n_estimators': 1304,
# 'depth': 8, 'fold_permutation_block': 142, 
# 'learning_rate': 0.21616891196578603, 
# 'od_pval': 0.12673190617341812, 
# 'l2_leaf_reg': 0.33021257848638497}

[I 2022-09-03 17:17:48,641] A new study created in memory with name: no-name-19c029b5-9264-4948-939b-ca64d788edd3


0:	learn: 0.3230750	total: 147ms	remaining: 1m
1:	learn: 0.1893608	total: 150ms	remaining: 30.9s
2:	learn: 0.1298416	total: 152ms	remaining: 20.8s
3:	learn: 0.0941040	total: 154ms	remaining: 15.8s
4:	learn: 0.0736168	total: 156ms	remaining: 12.8s
5:	learn: 0.0538523	total: 158ms	remaining: 10.8s
6:	learn: 0.0469055	total: 160ms	remaining: 9.33s
7:	learn: 0.0388139	total: 162ms	remaining: 8.26s
8:	learn: 0.0294966	total: 164ms	remaining: 7.42s
9:	learn: 0.0239922	total: 167ms	remaining: 6.74s
10:	learn: 0.0206200	total: 169ms	remaining: 6.19s
11:	learn: 0.0181142	total: 171ms	remaining: 5.73s
12:	learn: 0.0161914	total: 173ms	remaining: 5.35s
13:	learn: 0.0143809	total: 175ms	remaining: 5.02s
14:	learn: 0.0126586	total: 178ms	remaining: 4.74s
15:	learn: 0.0116261	total: 180ms	remaining: 4.5s
16:	learn: 0.0107543	total: 183ms	remaining: 4.28s
17:	learn: 0.0098313	total: 185ms	remaining: 4.08s
18:	learn: 0.0093508	total: 187ms	remaining: 3.9s
19:	learn: 0.0089742	total: 190ms	remaining: 3

[I 2022-09-03 17:17:49,751] Trial 0 finished with value: 1.0 and parameters: {'n_estimators': 415, 'depth': 10, 'fold_permutation_block': 76, 'learning_rate': 0.8305287485097345, 'od_pval': 0.9986172920629748, 'l2_leaf_reg': 1.7925301747404774}. Best is trial 0 with value: 1.0.


354:	learn: 0.0008967	total: 856ms	remaining: 145ms
355:	learn: 0.0008967	total: 858ms	remaining: 142ms
356:	learn: 0.0008967	total: 860ms	remaining: 140ms
357:	learn: 0.0008967	total: 862ms	remaining: 137ms
358:	learn: 0.0008967	total: 863ms	remaining: 135ms
359:	learn: 0.0008967	total: 865ms	remaining: 132ms
360:	learn: 0.0008967	total: 867ms	remaining: 130ms
361:	learn: 0.0008967	total: 869ms	remaining: 127ms
362:	learn: 0.0008967	total: 871ms	remaining: 125ms
363:	learn: 0.0008967	total: 873ms	remaining: 122ms
364:	learn: 0.0008967	total: 875ms	remaining: 120ms
365:	learn: 0.0008967	total: 877ms	remaining: 117ms
366:	learn: 0.0008967	total: 879ms	remaining: 115ms
367:	learn: 0.0008967	total: 881ms	remaining: 112ms
368:	learn: 0.0008967	total: 883ms	remaining: 110ms
369:	learn: 0.0008967	total: 885ms	remaining: 108ms
370:	learn: 0.0008966	total: 887ms	remaining: 105ms
371:	learn: 0.0008966	total: 888ms	remaining: 103ms
372:	learn: 0.0008966	total: 890ms	remaining: 100ms
373:	learn: 

[I 2022-09-03 17:17:58,450] Trial 1 finished with value: 1.0 and parameters: {'n_estimators': 1433, 'depth': 12, 'fold_permutation_block': 255, 'learning_rate': 0.7082338205556651, 'od_pval': 0.4385240269806403, 'l2_leaf_reg': 2.769004315191805}. Best is trial 0 with value: 1.0.


...

0:	learn: 0.3677873	total: 29.7ms	remaining: 33.4s
1:	learn: 0.2577572	total: 31.6ms	remaining: 17.7s
2:	learn: 0.2271518	total: 33ms	remaining: 12.3s
3:	learn: 0.1911173	total: 34.8ms	remaining: 9.73s
4:	learn: 0.1599422	total: 36.5ms	remaining: 8.17s
5:	learn: 0.1312393	total: 38.3ms	remaining: 7.14s
6:	learn: 0.1069633	total: 40.1ms	remaining: 6.39s
7:	learn: 0.0785147	total: 41.8ms	remaining: 5.83s
8:	learn: 0.0625298	total: 43.7ms	remaining: 5.41s
9:	learn: 0.0535771	total: 45.5ms	remaining: 5.07s
10:	learn: 0.0508318	total: 47.5ms	remaining: 4.8s
11:	learn: 0.0444030	total: 49.4ms	remaining: 4.57s
12:	learn: 0.0396217	total: 51.2ms	remaining: 4.37s
13:	learn: 0.0356908	total: 52.9ms	remaining: 4.19s
14:	learn: 0.0329857	total: 54.6ms	remaining: 4.03s
15:	learn: 0.0314486	total: 56.4ms	remaining: 3.9s
16:	learn: 0.0293333	total: 58.2ms	remaining: 3.79s
17:	learn: 0.0256297	total: 60.1ms	remaining: 3.69s
18:	learn: 0.0233995	total: 61.9ms	remaining: 3.6s
19:	learn: 0.0220624	total:

[I 2022-09-03 17:18:01,971] Trial 3 finished with value: 1.0 and parameters: {'n_estimators': 1123, 'depth': 9, 'fold_permutation_block': 248, 'learning_rate': 0.7120378027268229, 'od_pval': 0.1800642264669955, 'l2_leaf_reg': 1.9347728543933433}. Best is trial 0 with value: 1.0.


1071:	learn: 0.0010184	total: 1.78s	remaining: 84.5ms
1072:	learn: 0.0010183	total: 1.78s	remaining: 82.8ms
1073:	learn: 0.0010183	total: 1.78s	remaining: 81.2ms
1074:	learn: 0.0010182	total: 1.78s	remaining: 79.5ms
1075:	learn: 0.0010182	total: 1.78s	remaining: 77.9ms
1076:	learn: 0.0010182	total: 1.78s	remaining: 76.2ms
1077:	learn: 0.0010175	total: 1.78s	remaining: 74.5ms
1078:	learn: 0.0010175	total: 1.79s	remaining: 72.9ms
1079:	learn: 0.0010175	total: 1.79s	remaining: 71.2ms
1080:	learn: 0.0010175	total: 1.79s	remaining: 69.6ms
1081:	learn: 0.0010175	total: 1.79s	remaining: 67.9ms
1082:	learn: 0.0010175	total: 1.79s	remaining: 66.3ms
1083:	learn: 0.0010174	total: 1.79s	remaining: 64.6ms
1084:	learn: 0.0010174	total: 1.8s	remaining: 62.9ms
1085:	learn: 0.0010174	total: 1.8s	remaining: 61.3ms
1086:	learn: 0.0010174	total: 1.8s	remaining: 59.6ms
1087:	learn: 0.0010174	total: 1.8s	remaining: 58ms
1088:	learn: 0.0010174	total: 1.8s	remaining: 56.3ms
1089:	learn: 0.0010174	total: 1.8s	

[I 2022-09-03 17:18:18,490] Trial 4 finished with value: 1.0 and parameters: {'n_estimators': 2606, 'depth': 12, 'fold_permutation_block': 230, 'learning_rate': 0.08889036953870344, 'od_pval': 0.947828072893794, 'l2_leaf_reg': 1.8595032939224692}. Best is trial 0 with value: 1.0.


Best trial : score 1.0, 
params {'n_estimators': 415, 'depth': 10, 'fold_permutation_block': 76, 'learning_rate': 0.8305287485097345, 'od_pval': 0.9986172920629748, 'l2_leaf_reg': 1.7925301747404774}


optuna 라는 최적의 파라미터를 찾는 라이브러리를 통해 catboost의 파라미터 값을 찾았습니다.

In [19]:
# 훈련 
cat_paramets = {"learning_rate" : [0.01],
                'depth' : [8],
                'od_pval' : [0.12673190617341812],
                'fold_permutation_block': [142],
                'l2_leaf_reg' :[0.33021257848638497]}
cat = CatBoostClassifier(random_state=1127,verbose=False,n_estimators=1304)
model = RandomizedSearchCV(cat,cat_paramets,cv=kfold,n_jobs=-1,)
model.fit(x_train,y_train)   


NameError: name 'kfold' is not defined

In [ ]:
#예측 및 파일 저장
model.fit(x,y)
y_summit = model.predict(test_set)
y_summit = np.round(y_summit,0)
submission = pd.read_csv(path + 'sample_submission.csv',#예측에서 쓸거야!!
                      )
submission['ProdTaken'] = y_summit

submission.to_csv('test100.csv',index=False)